In [ ]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from f110_gym.envs.base_classes import Integrator
from collections import Counter,defaultdict
import random
import os
import pickle
import pandas as pd
from collections import defaultdict,deque
import math
import cmath
import scipy.stats as stats
import time
from sklearn.preprocessing import StandardScaler

In [ ]:
class Reward:
    def __init__(self, min_speed=0.8, max_speed=1.8, map_centers=None):
        """
        Initialize the Reward class.

        Parameters:
        min_speed (float): Minimum speed.
        max_speed (float): Maximum speed.
        map_centers (np.ndarray): Array of map center coordinates.
        """
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.map_centers = map_centers
        self.initial_point = np.array([[0, 0]])

        # Calculate total track length
        self.track_lengths = [np.linalg.norm(self.map_centers[i, :] - self.map_centers[i + 1, :]) for i in range(self.map_centers.shape[0] - 1)]
        self.total_track_length = np.sum(self.track_lengths)

        self.difference = np.diff(self.map_centers, axis=0)  # Calculate the difference between consecutive centers
        self.l2 = np.linalg.norm(self.difference, axis=1)    # Calculate the L2 norm of the difference

        # Hyperparameters
        self.track_width = 2.2
        self.epsilon = 1e-5
        self.distance_travelled = 0
        self.distance_multiplication_factor = 50
        self.distance_scaling_factor = 1.2

        self.min_distance = self.distance_multiplication_factor * ((np.exp(-0) / (2 - np.exp(-0))) - 0.7) + self.epsilon
        self.max_distance = self.distance_multiplication_factor * ((np.exp(-1) / (2 - np.exp(-1))) - 0.7) + self.epsilon

    def reset(self, point):
        """
        Reset the distance travelled and initial point.

        Parameters:
        point (np.ndarray): Initial point coordinates.
        """
        self.distance_travelled = 0
        self.initial_point = point

    def distance_reward(self, curr_position, next_position):
        """
        Calculate the distance reward.

        Parameters:
        curr_position (np.ndarray): Current position coordinates.
        next_position (np.ndarray): Next position coordinates.

        Returns:
        float: Distance reward.
        """
        distance = np.linalg.norm(curr_position - next_position)
        self.distance_travelled += distance
        return self.distance_travelled / self.total_track_length

    def calculate_distance_from_center(self, curr_x, curr_y):
        """
        Calculate the distance from the current position to the track centers.

        Parameters:
        curr_x (float): Current x-coordinate.
        curr_y (float): Current y-coordinate.

        Returns:
        tuple: Index of the closest center and the distance to it.
        """
        distances = np.linalg.norm(self.map_centers - np.array([curr_x, curr_y]), axis=1)
        return np.argmin(distances), distances[np.argmin(distances)]

    def centering_reward(self, x, y):
        """
        Calculate the centering reward.

        Parameters:
        x (float): Current x-coordinate.
        y (float): Current y-coordinate.

        Returns:
        float: Centering reward.
        """
        _, distance = self.calculate_distance_from_center(x, y)
        d = self.distance_multiplication_factor * ((np.exp(-distance) / (2 - np.exp(-distance))) - 0.6) + self.epsilon

        return self.distance_scaling_factor * d / (self.min_distance - self.max_distance)

    def calculate_reward(self, curr_state, next_state):
        """
        Calculate the total reward.

        Parameters:
        curr_state (np.ndarray): Current state coordinates.
        next_state (np.ndarray): Next state coordinates.

        Returns:
        float: Total reward.
        """
        distance_reward = self.distance_reward(curr_state, next_state)
        centering_reward = self.centering_reward(next_state[0], next_state[1])
        # print(f"Distance reward: {2*distance_reward}, Centering reward: {centering_reward}")
        return 2*distance_reward + centering_reward

In [ ]:
class IndexSelector:
    def __init__(self, num_indices):
        """
        Initialize the IndexSelector class.

        Parameters:
        num_indices (int): Number of indices.
        """
        self.num_indices = num_indices
        self.visited_indices = set()
        self.probabilities = np.ones(num_indices) / num_indices

    def select_index(self):
        """
        Select an index based on the current probabilities.

        Returns:
        int: Selected index.
        """
        if len(self.visited_indices) == self.num_indices:
            # Reset the probabilities and visited indices
            print('Visited all indices, resetting')
            self.visited_indices = set()
            self.probabilities = np.ones(self.num_indices) / self.num_indices

        # Select an index based on the current probabilities
        random_idx = np.random.choice(np.arange(self.num_indices), p=self.probabilities)

        # Update the probabilities
        self.visited_indices.add(random_idx)
        if len(self.visited_indices) < self.num_indices:
            self.probabilities[random_idx] = 0
            remaining_prob = 1 - np.sum(self.probabilities)
            self.probabilities[self.probabilities > 0] += remaining_prob / np.sum(self.probabilities > 0)

        return random_idx

In [ ]:
class F1Tenth_navigation:
    """
    A class to navigate the F1Tenth environment using SARSA(λ) and Expected SARSA algorithms.

    Attributes:
        env (gym.Env): The gym environment.
        sx, sy, stheta (float): Starting x, y coordinates and orientation.
        save_path (str): Path to save the weights.
        track_name (str): Name of the track.
        num_agents (int): Number of agents.
        map_path (str): Path to the map.
        map_ext (str): Extension of the map file.
        map_centers (np.ndarray): Array of map center coordinates.
        index_selector (IndexSelector): Index selector for resetting positions.
        random_seed (int): Random seed for reproducibility.
        num_beams (int): Number of LiDAR beams.
        n_features (int): Number of binary features.
        binary_powers (np.ndarray): Powers of 2 for binary conversion.
        angle (int): Angle for LiDAR.
        num_states (int): Number of states.
        num_actions (int): Number of actions.
        angles_deg (np.ndarray): Array of angles in degrees.
        angles_rad (np.ndarray): Array of angles in radians.
        min_speed, max_speed (float): Minimum and maximum speed.
        mu, sig (float): Mean and standard deviation for Gaussian speed distribution.
        action_space (dict): Action space mapping.
        weights (np.ndarray): Q-table weights.
        ET (np.ndarray): Eligibility trace.
        IS (np.ndarray): Instructive signal.
        discount_factor (float): Discount factor for SARSA.
        learning_rate (float): Learning rate for SARSA.
        decay_rate (float): Decay rate for eligibility trace.
        IS_decay_rate (float): Decay rate for instructive signal.
        epsilon_treshold (float): Epsilon threshold for epsilon-greedy policy.
        numerical_stability (float): Numerical stability constant.
        reward_class (Reward): Reward calculation class.
        reward, cumulative_reward (float): Reward values.
        episode_reward (list): List of episode rewards.
        curr_state (int): Current state index.
        curr_action_index (int): Current action index.
        collision_count (int): Collision count.
        distance_threshold (float): Distance threshold for reset.
        state_counter (np.ndarray): Counter for state visits.
        reset_position (np.ndarray): Reset position coordinates.
        current_lap (int): Current lap count.
        visited_states (set): Set of visited states.
        visited_indices (set): Set of visited indices.
    """

    def __init__(self, gym_env_code='f110_gym:f110-v0', num_agents=1, map_path='./f1tenth_racetracks/Austin/Austin_map', map_ext='.png', sx=0., sy=0., stheta=0., map_centers_file=None, save_path=None, track_name=None, inference=None):
        """
        Initialize the F1Tenth_navigation class.

        Parameters:
            gym_env_code (str): Gym environment code.
            num_agents (int): Number of agents.
            map_path (str): Path to the map.
            map_ext (str): Extension of the map file.
            sx, sy, stheta (float): Starting x, y coordinates and orientation.
            map_centers_file (str): Path to the map centers file.
            save_path (str): Path to save the weights.
            track_name (str): Name of the track.
            inference (str): Path to the inference weights file.
        """
        self.env = gym.make(gym_env_code, map=map_path, map_ext=map_ext, num_agents=num_agents, timestep=0.01, integrator=Integrator.RK4)
        self.env.add_render_callback(self.render_callback)
        self.sx, self.sy, self.stheta = sx, sy, stheta
        self.save_path = save_path
        self.track_name = track_name
        self.num_agents = num_agents
        self.map_path = map_path
        self.map_ext = map_ext
        self.map_centers_file = pd.read_csv(map_centers_file)
        self.map_centers_file.columns = ['x', 'y', 'w_r', 'w_l']
        self.map_centers_file.index = self.map_centers_file.index.astype(int)
        self.map_centers = self.map_centers_file.values[:, :2]

        self.index_selector = IndexSelector(self.map_centers.shape[0])
        self.scaler = StandardScaler()

        self.random_seed = 42
        np.random.seed(self.random_seed)

        self.num_beams = 1080
        self.n_features = 10
        self.binary_powers = np.array([2 ** i for i in range(self.n_features)])
        self.angle = 270

        self.num_states = 2 ** self.n_features
        self.num_actions = 51
        

        self.angles_deg = np.linspace(-self.angle // 2, self.angle // 2, self.num_actions)[::-1]
        self.angles_rad = self.convert_deg_to_rad(self.angles_deg)

        self.min_speed = 0.5
        self.max_speed = 1.8
        self.mu = self.num_actions // 2
        self.sig = self.num_actions // 4

        # self.speeds = np.linspace(self.min_speed, self.max_speed, self.num_actions)
        self.action_space = self.generate_action_space(self.angles_rad)

        if inference is not None:
            self.weights = np.load(inference)
            print(f'Loaded already trained weights')
            self.collision_count = int(inference.split('_')[1].split('.')[0])
        else:
            self.weights = np.zeros((self.num_states, self.num_actions))
            self.collision_count = 0
        
        self.ET_IS = np.zeros((self.num_states, self.num_actions))

        # self.ET = np.zeros((self.num_states, 1))
        # self.IS = np.zeros((1, self.num_actions))

        self.discount_factor = 0.95
        self.learning_rate = 0.1
        self.decay_rate = 0.9
        self.IS_decay_rate = 0.7
        self.epsilon_decay_rate=0.99997

        self.epsilon_treshold = 0.25
        self.numerical_stability = 1e-5

        self.reward_class = Reward(min_speed=self.min_speed, max_speed=self.max_speed, map_centers=self.map_centers)
        self.reward = 0
        self.cumulative_reward = 0
        self.episode_reward = []
        self.multi_episode_reward=[]

        self.curr_state = None
        self.curr_action_index = None

        self.distance_threshold = 0.5
        self.state_counter = np.zeros(self.num_states)
        self.reset_position = np.array([[self.sx, self.sy]])

        self.current_lap = 1
        self.visited_states = set()
        self.visited_indices = set()

    def render_callback(self, env_renderer):
        """
        Render callback function to update the camera to follow the car.

        Parameters:
            env_renderer (EnvRenderer): The environment renderer.
        """
        e = env_renderer
        x = e.cars[0].vertices[::2]
        y = e.cars[0].vertices[1::2]
        top, bottom, left, right = max(y), min(y), min(x), max(x)
        e.score_label.x = left
        e.score_label.y = top - 700
        e.left = left - 800
        e.right = right + 800
        e.top = top + 800
        e.bottom = bottom - 800

    def convert_deg_to_rad(self, array):
        """
        Convert an array of angles from degrees to radians.

        Parameters:
            array (np.ndarray): Array of angles in degrees.

        Returns:
            np.ndarray: Array of angles in radians.
        """
        return array * np.pi / 180

    def gaussian_speed(self, location):
        """
        Calculate the Gaussian speed for a given location.

        Parameters:
            location (int): Location index.

        Returns:
            float: Gaussian speed.
        """
        speed = (1 / (self.sig * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((location - self.mu) / self.sig) ** 2)
        scale_factor = self.max_speed / (1 / (self.sig * np.sqrt(2 * np.pi)))
        scaled_value = speed * scale_factor
        return np.round(np.clip(scaled_value, self.min_speed, self.max_speed), 4)

    def generate_action_space(self, angles):
        """
        Generate the action space mapping.

        Parameters:
            angles (np.ndarray): Array of angles in radians.

        Returns:
            dict: Action space mapping.
        """
        action_space = {}
        for idx, angle in enumerate(angles):
            # action_space[idx] = (angle, self.gaussian_speed(idx))
            action_space[idx] = (angle, 1.0)
        return action_space

    def lidar_to_binary_features(self, lidar_input, n_features=10, n_sectors=36, threshold_percentile=40):
        """
        Convert LiDAR scan of 1080 values to a binary representation of 10 bits.

        Parameters:
            lidar_input (np.ndarray): LiDAR input array of shape (1080,).
            n_features (int): Number of binary features to generate (default: 10).
            n_sectors (int): Number of sectors to divide scan into (default: 36).
            threshold_percentile (int): Percentile for thresholding (default: 50).

        Returns:
            np.ndarray: Binary features array of shape (n_features,).
        """
        rng = np.random.RandomState(self.random_seed)
        lidar_input = np.asarray(lidar_input, dtype=np.float32)

        sector_size = lidar_input.shape[0] // n_sectors
        sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)

        sector_features = np.concatenate([
            np.mean(sectors, axis=1),
            np.std(sectors, axis=1),
            np.max(sectors, axis=1),
            np.min(sectors, axis=1),
            np.median(sectors, axis=1),
            np.percentile(sectors, 75, axis=1)
        ])

        gradients = []
        for scale in [1, 2, 3]:
            rolled = np.roll(sector_features, scale)
            gradient = sector_features - rolled
            gradients.append(gradient)
        gradient_features = np.concatenate(gradients)

        combined_features = np.concatenate([sector_features, gradient_features, np.abs(gradient_features)])
        combined_features = (combined_features - np.mean(combined_features)) / (np.std(combined_features) + 1e-8)

        input_dim = combined_features.shape[0]
        projection_matrix = rng.normal(0, 1, (input_dim, n_features))
        projection_matrix /= np.linalg.norm(projection_matrix, axis=0) + 1e-8

        projected_features = np.dot(combined_features, projection_matrix)
        thresholds = np.percentile(projected_features, threshold_percentile, axis=0)

        temperature = 0.7
        z_scores = (projected_features - thresholds) / temperature
        probabilities = stats.norm.cdf(z_scores)

        random_matrix = rng.uniform(0, 1, probabilities.shape)
        binary_features = (probabilities >= random_matrix).astype(np.int8)

        return binary_features

    # def lidar_to_binary_features(self,lidar_input, scaler,binary_threshold=0.0,n_sectors=12):
    #     """
    #     Convert LiDAR input into a binary representation using statistical features.

    #     Parameters:
    #     - lidar_input: 1D array of LiDAR values.
    #     - n_sectors: Number of sectors to divide the input into.

    #     Returns:
    #     - binary_code: 1D array of binary representation.
    #     """
    #     lidar_input = np.asarray(lidar_input, dtype=np.float32)
    #     sector_size = lidar_input.shape[0] // n_sectors
    #     sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)
    #     sector_features = np.vstack(
    #         [   np.mean(sectors, axis=1),
    #             np.std(sectors, axis=1),
    #             np.max(sectors, axis=1),
    #             np.min(sectors, axis=1),
    #             np.median(sectors, axis=1),
    #             np.percentile(sectors, 75, axis=1)
    #         ])
    #     scaled_features = scaler.partial_fit(sector_features.T).transform(sector_features.T).T
    #     binary_features = (scaled_features > binary_threshold).astype(int)

    #     binary_code = (binary_features.sum(axis=0) > 0).astype(int)  
    #     return np.array(binary_code).reshape(1,-1)

    def vectorized_10bit_lsh(self,x, random_state):
            """
            Vectorized LSH bit generation.
            """
            # Projection matrix generation
            input_dim = len(x)
            n_projections = max(20, input_dim)
            
            # Vectorized Cauchy projection
            projection_matrix = random_state.standard_cauchy((input_dim, n_projections))
            
            # Normalize projection matrix columns
            projection_matrix /= np.linalg.norm(projection_matrix, axis=0, keepdims=True)
            
            # Vectorized projection
            projected = x @ projection_matrix
            
            # Compute thresholds
            thresholds = np.median(projected, axis=0)
            
            # Vectorized bit generation
            bit_matrix = (projected >= thresholds).astype(np.int8)
            
            # Vectorized 10-bit generation using rolling XOR
            # Create a sliding window of projections
            sliding_window = np.lib.stride_tricks.sliding_window_view(bit_matrix.T, 2)
            
            # Compute XOR across sliding windows
            xor_bits = np.bitwise_xor(sliding_window[..., 0], sliding_window[..., 1])
            
            # Take first 10 bits
            return xor_bits[:self.n_features]
    
    # def lidar_to_binary_features(self, lidar_input, n_features=10, n_sectors=36):
    #     """
    #     Convert LiDAR scan to binary features using Locality-Sensitive Hashing.
        
    #     Parameters:
    #         lidar_input (np.ndarray): LiDAR input array
    #         n_features (int): Number of binary features to generate
    #         n_sectors (int): Number of sectors to divide scan into
        
    #     Returns:
    #         np.ndarray: Binary features array
    #     """
    #     lidar_input = np.asarray(lidar_input, dtype=np.float32)
        
    #     # Reshape into sectors
    #     sector_size = lidar_input.shape[0] // n_sectors
    #     sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)
        
    #     # Vectorized statistical features
    #     sector_stats = np.column_stack([
    #         np.mean(sectors, axis=1),
    #         np.std(sectors, axis=1),
    #         np.max(sectors, axis=1),
    #         np.min(sectors, axis=1),
    #         np.median(sectors, axis=1),
    #         np.percentile(sectors, 75, axis=1)
    #         ])
        
    #     # Vectorized gradient computation
    #     scales = np.array([1, 2, 3])
    #     rolled_features = np.stack([
    #         np.roll(sector_stats, shift) for shift in scales
    #     ])
        
    #     # Compute gradient variations
    #     gradient_ops = np.stack([
    #         sector_stats - rolled_features,  # difference
    #         np.abs(sector_stats - rolled_features),  # absolute difference
    #         np.square(sector_stats - rolled_features)  # squared difference
    #     ])
        
    #     # Flatten and combine features
    #     combined_features = np.concatenate([
    #         sector_stats.ravel(),
    #         gradient_ops.ravel()
    #     ])
        
    #     # Normalized features
    #     features = (combined_features - np.mean(combined_features)) / (np.std(combined_features) + 1e-8)
    #     rng = np.random.RandomState(self.random_seed)
    #     # Generate binary features
    #     binary_features = self.vectorized_10bit_lsh(features, rng)
    #     return np.array(binary_features).reshape(1,-1)
    
    def get_state(self, feature):
        """
        Get the state index from the binary feature vector.

        Parameters:
            feature (np.ndarray): Binary feature vector.

        Returns:
            int: State index.
        """
        return np.dot(feature[0], self.binary_powers)

    def select_action(self, state_idx):
        """
        Select an action based on the epsilon-greedy policy.

        Parameters:
            state_idx (int): State index.

        Returns:
            tuple: Action index and action (steering angle, speed).
        """
        random_number = np.random.rand()
        action_idx = None

        if random_number <= self.epsilon_treshold:
            action_idx = np.random.choice(list(self.action_space.keys()))
        else:
            max_value = np.max(self.weights[state_idx])
            max_indices = np.where(np.abs(self.weights[state_idx] == max_value))[0]
            action_idx = np.random.choice(max_indices)

        self.epsilon_treshold *= self.epsilon_decay_rate
        return action_idx, self.action_space[action_idx]

    def inference_action(self, state_idx):
        """
        Select the action with the highest Q-value for inference.

        Parameters:
            state_idx (int): State index.

        Returns:
            tuple: Action index and action (steering angle, speed).
        """
        action_idx = np.argmax(self.weights[state_idx])
        return action_idx, self.action_space[action_idx]

    def normalize_weights(self,values, neg_lower_bound=-3, neg_upper_bound=-0.1, pos_lower_bound=0.1, pos_upper_bound=3):
        values = np.array(values)
    
        pos_mask = values > 0
        neg_mask = values < 0
        
        pos_values = values[pos_mask]
        neg_values = values[neg_mask]
        
        normalized_values = np.zeros_like(values, dtype=float)
        
        if pos_values.size > 0:
            pos_min = pos_values.min()
            pos_max = pos_values.max()
            if pos_min != pos_max:
                pos_normalized = (pos_values - pos_min) / (pos_max - pos_min)
                pos_scaled = pos_lower_bound + (pos_upper_bound - pos_lower_bound) * pos_normalized
            else:
                pos_scaled = np.full(pos_values.shape, pos_lower_bound)
            normalized_values[pos_mask] = pos_scaled
        
        if neg_values.size > 0:
            neg_min = neg_values.min()
            neg_max = neg_values.max()
            if neg_min != neg_max:
                neg_normalized = (neg_values - neg_min) / (neg_max - neg_min)
                neg_scaled = neg_lower_bound + (neg_upper_bound - neg_lower_bound) * neg_normalized
            else:
                neg_scaled = np.full(neg_values.shape, neg_upper_bound)
            normalized_values[neg_mask] = neg_scaled
        
        return np.round(normalized_values,4)

    def expected_sarsa_weight_update(self, next_state, reward):
        """
        Update the weights using the Expected SARSA algorithm.

        Parameters:
            next_state (int): Next state index.
            reward (float): Reward received.
        """
        # self.ET[self.curr_state, :] = 1
        # self.IS[:, self.curr_action_index] = 1
        self.ET_IS[self.curr_state, self.curr_action_index] = 1

        q_values = self.weights[next_state, :]
        q_max = np.max(q_values)
        greedy_actions = np.sum(q_values == q_max)

        non_greedy_action_probability = self.epsilon_treshold / self.num_actions
        greedy_action_probability = ((1 - self.epsilon_treshold) / greedy_actions) + non_greedy_action_probability

        greedy_mask = (q_values == q_max)
        non_greedy_mask = ~greedy_mask

        expected_q = np.sum(q_values[greedy_mask] * greedy_action_probability) + np.sum(q_values[non_greedy_mask] * non_greedy_action_probability)

        delta = reward + self.discount_factor * expected_q - self.weights[self.curr_state, self.curr_action_index]

        # self.weights += self.learning_rate * delta * (self.ET @ self.IS)
        self.weights += self.learning_rate * delta * self.ET_IS
        self.weights = self.normalize_weights(self.weights)
        self.decay_ET_IS()

    def sarsa_lambda_weight_update(self, next_state, reward):
        """
        Update the weights using the SARSA(λ) algorithm.

        Parameters:
            next_state (int): Next state index.
            reward (float): Reward received.
        """
        # self.ET[self.curr_state, :] = 1
        # self.IS[:, self.curr_action_index] = 1
        self.ET_IS[self.curr_state, self.curr_action_index] = 1

        next_action_index, _ = self.select_action(next_state)
        delta = reward + self.discount_factor * self.weights[next_state, next_action_index] - self.weights[self.curr_state, self.curr_action_index]

        # self.weights += self.learning_rate * delta * (self.ET @ self.IS)
        self.weights += self.learning_rate * delta * self.ET_IS
        self.weights = self.normalize_weights(self.weights)
        self.decay_ET_IS()

    def decay_ET_IS(self):
        """
        Decay the eligibility trace and instructive signal.
        """
        # self.ET *= self.discount_factor * self.decay_rate
        # self.IS *= self.discount_factor * self.IS_decay_rate
        self.ET_IS *= self.discount_factor * self.decay_rate

        # self.ET = np.round(self.ET, 4)
        # self.IS = np.round(self.IS, 4)
        # self.ET_IS = np.round(self.ET_IS, 4)

    def reset_ET_IS(self):
        """
        Reset the eligibility trace and instructive signal.
        """
        # self.ET.fill(0)
        # self.IS.fill(0)
        self.ET_IS.fill(0)

    def train(self):
        """
        Train the agent using the Expected SARSA algorithm.
        """
        obs, step_reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.stheta]]))
        lidar = obs['scans'][0]
        # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
        down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
        self.curr_state = self.get_state(down_sampled)
        episode_reward = 0
        self.reset_position = np.array([[self.sx, self.sy]])
        self.reward_class.reset(self.reset_position)
        start_time=time.time()
        while True:
            if self.curr_state not in self.visited_states:
                self.visited_states.add(self.curr_state)

            print(f'Downsampled is {down_sampled} and curr state is {self.curr_state}')
            self.curr_action_index, (steering_angle, speed) = self.select_action(self.curr_state)
            print(f'Action index is {self.curr_action_index} and action is {steering_angle, speed}')
            curr_x = obs['poses_x'][0]
            curr_y = obs['poses_y'][0]
            obs, reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
            down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
            next_state = self.get_state(down_sampled)

            if done:
                self.reward = -100
            else:
                self.reward = self.reward_class.calculate_reward(np.array([curr_x, curr_y]), np.array([obs['poses_x'][0], obs['poses_y'][0]]))

            self.cumulative_reward += self.reward
            episode_reward += self.reward

            self.expected_sarsa_weight_update(next_state, self.reward)
            # self.sarsa_lambda_weight_update(next_state, self.reward)
            self.curr_state = next_state

            if done:
                random_idx = self.index_selector.select_index()
                n_x, n_y = self.map_centers[random_idx]
                delta_x, delta_y = np.random.uniform(-0.75, 0.75), np.random.uniform(-0.2, 0.2)
                n_theta = np.random.choice(self.angles_rad)
                obs, step_reward, done, info = self.env.reset(np.array([[n_x + delta_x, n_y + delta_y, n_theta]]))
                lidar = obs['scans'][0]
                # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
                down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
                self.curr_state = self.get_state(down_sampled)

                self.reset_position = np.array([[n_x + delta_x, n_y + delta_y]])
                self.reward_class.reset(self.reset_position)
                self.reset_ET_IS()
                self.episode_reward.append(episode_reward)
                episode_reward = 0
                # print(f'Cumulative is {self.cumulative_reward} and normal is {self.episode_reward}')
                if (self.collision_count + 1) % 1000 == 0:
                    end_time=time.time()
                    self.multi_episode_reward.append(sum(self.episode_reward))
                    print(f'Episode:{self.collision_count + 1}, Time:{end_time-start_time} reward:{sum(self.episode_reward)}, Cumulative Reward:{self.cumulative_reward}, observed states:{len(self.visited_states)}')
                    self.episode_reward = []
                    start_time=end_time
                    if not os.path.exists(os.path.join(self.save_path, self.track_name)):
                        os.mkdir(os.path.join(self.save_path, self.track_name))
                    np.save(os.path.join(self.save_path, self.track_name, f'rewards.npy'), np.array(self.multi_episode_reward))
                    
                if (self.collision_count + 1) % 5000 == 0:
                    if not os.path.exists(os.path.join(self.save_path, self.track_name)):
                        os.mkdir(os.path.join(self.save_path, self.track_name))
                    np.save(os.path.join(self.save_path, self.track_name, f'weights_{self.collision_count + 1}.npy'), self.weights)
                    print(f'File saved')
                self.collision_count += 1

            self.env.render(mode='human')

    def inference(self):
        """
        Inference the agent using the Expected SARSA algorithm.
        """
        obs, reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.stheta]]))
        lidar = obs['scans'][0]
        # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
        down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
        self.curr_state = self.get_state(down_sampled)

        while not done:
            _, (steering_angle, speed) = self.inference_action(self.curr_state)
            obs, reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
            down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
            self.curr_state = self.get_state(down_sampled)
            
            if done:
                random_idx = self.index_selector.select_index()
                n_x, n_y = self.map_centers[random_idx]
                delta_x, delta_y = np.random.uniform(-0.75, 0.75), np.random.uniform(-0.2, 0.2)
                n_theta = np.random.choice(self.angles_rad)
                obs, step_reward, done, info = self.env.reset(np.array([[n_x + delta_x, n_y + delta_y, n_theta]]))
                lidar = obs['scans'][0]
                # down_sampled = self.lidar_to_binary_features(lidar,self.scaler,n_sectors=self.n_features).reshape(1, -1)
                down_sampled = self.lidar_to_binary_features(lidar).reshape(1,-1)
                self.curr_state = self.get_state(down_sampled)

                self.reset_position = np.array([[n_x + delta_x, n_y + delta_y]])
                self.reward_class.reset(self.reset_position)
                self.reset_ET_IS()
                self.collision_count+=1

            self.env.render(mode='human')


In [ ]:
# path = '/workspaces/F1_tenth_training/f1tenth_gym_ros/f1tenth_racetracks'
path = './f1tenth_racetracks'
all_map_paths=[]
map_centers = []
map_names = []
track_lengths=[]
for folder in os.listdir(path):
    if folder not in ['README.md','.gitignore','convert.py','LICENSE','rename.py','.git']:
        folder_name=folder
        file_name=folder_name.replace(' ','')+'_map'
        map_center = folder_name.replace(' ','')+'_centerline.csv'
        track_lengths.append(len(pd.read_csv(f'{path}/{folder_name}/{map_center}')))
        map_names.append(folder_name)
        all_map_paths.append(f'{path}/{folder_name}/{file_name}')
        map_centers.append(f'{path}/{folder_name}/{map_center}')

list(zip(map_names,track_lengths))

In [ ]:
global num_agents,map_path,map_ext,sx,sy,stheta
num_agents = 1
map_ext = '.png'
sx = 0.
sy = 0.
stheta = 1.0
map_path = all_map_paths[0]
map_center = map_centers[0]
map_name = map_names[0]
save_path = './Weights/'
inference_file = None
# inference_file = './Weights/Montreal/weights_55000.npy'
simulator = F1Tenth_navigation(num_agents=num_agents,map_path=map_path,map_ext=map_ext,sx=sx,sy=sy,stheta=stheta,map_centers_file=map_center,save_path=save_path,track_name=map_name,inference=inference_file)
simulator.train()
# simulator.inference()

np.save(f'{save_path}{map_name}/weights_4000.npy',simulator.weights)

In [ ]:
simulator.multi_episode_reward